In [ ]:
f = open('results_tensor2.csv','r')
X_data = []
y_data = []
for line in f:
    data = line.strip().split(',')
    X_data.append(data[:56])
    y_data.append(data[56:])
X_train = X_data[:5000]
y_train = y_data[:5000]
X_test = X_data[5000:]
y_test = y_data[5000:]


In [9]:
import sys
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.visualize_util import plot
from keras.layers import Merge, merge
from keras.layers import Input


import numpy
numpy.set_printoptions(threshold=numpy.nan)

eng_num_preps = 14
ger_num_preps = 10
PCA_dimension = 56
language_embedding_dimension = 28

eng_num_preps = 2
ger_num_preps = 3
PCA_dimension = 3
language_embedding_dimension = 4


def load_english_data():
	''' 1. seperate train and test better
		2. change number of iterations for test/train
	'''
	f = open('results_tensor2.csv','r')
	XY_data=[]
	X_data = []
	y_data = []
	second_language = [0 for x in range(ger_num_preps)]
	for line in f:
	    data = line.strip().split(',') + [0] + second_language
	    XY_data.append(data)
	    # add one to signify what language
	    # 0 = english
	    # 1 = german
	    X_data.append(data[:PCA_dimension+1])
	    y_data.append(data[PCA_dimension+1:])
	X_train = X_data[:5000]
	y_train = y_data[:5000]
	X_test = X_data[5000:]
	y_test = y_data[5000:]
	return X_train,y_train, X_test, y_test

def load_fake_data():
	''' 1. seperate train and test better
		2. change number of iterations for test/train
	'''
	f = open('fake_tensor_results.csv','r')
	XY_data=[]
	X_data = []
	y_data = []
	second_language = [0 for x in range(ger_num_preps)]
	for line in f:
	    data = line.strip().split(',') + [0] + second_language
	    XY_data.append(data)
	    # add one to signify what language
	    # 0 = english
	    # 1 = german
	    X_data.append(data[:PCA_dimension+1])
	    y_data.append(data[PCA_dimension+1:])
	X_train = X_data[:10]
	y_train = y_data[:10]
	X_test = X_data[10:]
	y_test = y_data[10:]
	return X_train,y_train, X_test, y_test

def set_fake_pca_weights():
	'''expects a file with weight on a each new line '''
	pca_results = open('results_fake_PCA.csv')
	pca_weights = np.zeros((PCA_dimension+1,PCA_dimension+1))
	i = 0
	for line in pca_results:
		a = float(line.strip())
		pca_weights[i][i] = a
		i +=1
	pca_weights[i][i] = 1.0
	# unsure what to set the language gate to start at
	# currently set to 1
	bias = np.concatenate((np.zeros(PCA_dimension),np.ones(1)))
	return [pca_weights,bias]

# X_train, y_train, X_test, y_test = load_english_data()
#
X_train, y_train, X_test, y_test = load_fake_data()

# unique_data = [list(x) for x in set(tuple(x) for x in XY_data)]
# print len(unique_data)
# this shows that there are only a total of 101 unique X-Y pairs

def set_pca_weights():
	'''expects a file with weight on a each new line '''
	pca_results = open('results_PCA.csv')
	pca_weights = np.zeros((PCA_dimension+1,PCA_dimension+1))
	i = 0
	for line in pca_results:
		a = float(line.strip())
		pca_weights[i][i] = a
		i +=1
	pca_weights[i][i] = 1.0
	# unsure what to set the language gate to start at
	# currently set to 1
	bias = np.concatenate((np.zeros(PCA_dimension),np.ones(1)))
	return [pca_weights,bias]

# _______________first layer___________________#
# language_input = Sequential()
language_input = Input(shape=(PCA_dimension+1,))
# gets the pca over all languages to initialize gates to

# gate_weights = set_pca_weights()
gate_weights = set_fake_pca_weights()

gate_layer = Dense(PCA_dimension+1, activation='linear', input_dim=PCA_dimension+1, weights=gate_weights)(language_input)
# _____________________________________________#

# _____________English Middle Layer____________#
english_representation = Dense(language_embedding_dimension,activation='linear', init='uniform')(gate_layer)

# _____________________________________________#

# _____________English Model_____________#
english_predictions = Dense(eng_num_preps+ger_num_preps, init='uniform', activation='softmax')(english_representation)
english_model = Model(name='english', input=language_input, output=english_predictions)
# _____________________________________________#


# ____________Optimization & Training___________#
sgd = SGD(lr=0.1, decay=1e-6)
english_model.compile(loss='mean_squared_error',
              optimizer=sgd,
              metrics=['accuracy'])

english_model.fit(X_train, y_train,
          nb_epoch=1,
          batch_size=1)
score = english_model.evaluate(X_test, y_test, batch_size=10)
# _____________________________________________#


german_representation = Dense(language_embedding_dimension,activation='linear', init='uniform')(gate_layer)
bilingual_representation = merge([english_representation,german_representation], mode='concat')

# german_predictions = Dense(eng_num_preps+ger_num_preps, init='uniform', activation='softmax')(bilingual_representation)

# bilingual_predictions = Dense(eng_num_preps+ger_num_preps, weights=english_predictions+german_predictions, activation='softmax')
bilingual_predictions= Dense(eng_num_preps+ger_num_preps, init='uniform', activation='softmax')(bilingual_representation)
bilingual_model = Model(name='bilingual', input=language_input, output=bilingual_predictions)


Epoch 1/1
10/10 [==============================] - 0s


In [42]:
print dir(english_predictions.transfer)



['__call__', '__class__', '__cmp__', '__delattr__', '__doc__', '__format__', '__func__', '__get__', '__getattribute__', '__hash__', '__init__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'im_class', 'im_func', 'im_self']


In [18]:
def print_weights(models):
	for model in models:
		weights = model.get_weights()
		filename = 'keras_' + model.name + '_weights.txt'
		g = open(filename,'w')
		g.write(str(weights))
		g.close()

	# weights = bilingual_model.get_weights()
	# g = open('keras_bilingual_weights.txt','w')
	# g.write(str(weights))
	# g.close()
print_weights([english_model, bilingual_model])